In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plot
import seaborn as sns
import pickle

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression, LassoCV, RidgeCV
from sklearn.svm import LinearSVR
from sklearn.ensemble import AdaBoostRegressor, RandomForestRegressor

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import Pipeline

### Cleaning and EDA

In [3]:
df = pd.read_csv('../data/calls_weather_tfk.csv')

In [4]:
df.head()

,Unnamed: 0,year,month,day,hour,num_calls,BRONX,BROOKLYN,MANHATTAN,QUEENS,...,NAME,DATE,AWND,PRCP,SNOW,SNWD,TMAX,TMIN,TAVG_CALC,Incidences
0,0,2010,1,2,2,93,25,23,20,20,...,"NY CITY CENTRAL PARK, NY US",2010-01-02,12.53,0.02,0.2,0.0,34.0,17.0,25.5,1
1,1,2010,1,2,3,88,22,28,19,15,...,"NY CITY CENTRAL PARK, NY US",2010-01-02,12.53,0.02,0.2,0.0,34.0,17.0,25.5,1
2,2,2010,1,3,12,144,22,48,36,34,...,"NY CITY CENTRAL PARK, NY US",2010-01-03,14.32,0.00,0.0,0.0,22.0,17.0,19.5,1
3,3,2010,1,4,1,94,23,34,20,11,...,"NY CITY CENTRAL PARK, NY US",2010-01-04,10.74,0.00,0.0,0.0,30.0,19.0,24.5,2
4,4,2010,1,4,10,219,53,67,54,34,...,"NY CITY CENTRAL PARK, NY US",2010-01-04,10.74,0.00,0.0,0.0,30.0,19.0,24.5,2


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25465 entries, 0 to 25464
Data columns (total 23 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Unnamed: 0                25465 non-null  int64  
 1   year                      25465 non-null  int64  
 2   month                     25465 non-null  int64  
 3   day                       25465 non-null  int64  
 4   hour                      25465 non-null  int64  
 5   num_calls                 25465 non-null  int64  
 6   BRONX                     25465 non-null  int64  
 7   BROOKLYN                  25465 non-null  int64  
 8   MANHATTAN                 25465 non-null  int64  
 9   QUEENS                    25465 non-null  int64  
 10  RICHMOND / STATEN ISLAND  25465 non-null  int64  
 11  UNKNOWN                   25465 non-null  int64  
 12  STATION                   25465 non-null  object 
 13  NAME                      25465 non-null  object 
 14  DATE  

Unnecessary columns: Unnamed: 0 (extraneous), STATION, NAME (ID and name of Central Park weather station), Date (already have year, month, day, and hour columns)

Nulls: AWND (average wind speed) has null values

Correlated columns: Boroughs add up to num_calls, TAVG_CALC (avg temp) is the average of TMAX (max temp) and TMIN (min temp)

In [24]:
# Check correlation between TMAX and TAVG
df['TMAX'].corr(df['TAVG_CALC'])

0.9905403528938205

In [25]:
# Check correlation between TMIN and TAVG
df['TMIN'].corr(df['TAVG_CALC'])

0.9887547613202625

In [26]:
# Check correlation between TMAX and TMIN
df['TMAX'].corr(df['TMIN'])

0.9588805187703731

In [6]:
# Drop columns as needed
df.drop(columns=['Unnamed: 0','STATION','NAME','DATE','AWND', 'TMAX', 'TMIN'],inplace=True)

In [28]:
df

,year,month,day,hour,num_calls,BRONX,BROOKLYN,MANHATTAN,QUEENS,RICHMOND / STATEN ISLAND,UNKNOWN,PRCP,SNOW,SNWD,TAVG_CALC,Incidences
0,2010,1,2,2,93,25,23,20,20,5,0,0.02,0.2,0.0,25.5,1
1,2010,1,2,3,88,22,28,19,15,4,0,0.02,0.2,0.0,25.5,1
2,2010,1,3,12,144,22,48,36,34,4,0,0.00,0.0,0.0,19.5,1
3,2010,1,4,1,94,23,34,20,11,6,0,0.00,0.0,0.0,24.5,2
4,2010,1,4,10,219,53,67,54,34,11,0,0.00,0.0,0.0,24.5,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25460,2016,8,23,21,213,51,55,55,44,8,0,0.00,0.0,0.0,71.5,8
25461,2016,8,23,22,176,54,44,42,29,7,0,0.00,0.0,0.0,71.5,11
25462,2016,8,23,23,152,35,43,39,31,4,0,0.00,0.0,0.0,71.5,8
25463,2016,8,23,12,187,41,53,48,36,9,0,0.00,0.0,0.0,71.5,8


In [7]:
# Set X and y
X = df.drop(columns=['num_calls','BRONX','BROOKLYN','MANHATTAN','QUEENS','RICHMOND / STATEN ISLAND','UNKNOWN'])
y = df['num_calls']

In [8]:
# Train test split
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=42)

In [9]:
# Scale Xs
sc = StandardScaler()
X_train_sc = sc.fit_transform(X_train)
X_test_sc = sc.transform(X_test)

### Linear Regression

In [128]:
# instantiate, fit, and score
lr = LinearRegression()
lr.fit(X_train_sc, y_train)
lr.score(X_train_sc, y_train), lr.score(X_test_sc, y_test)

(0.5143614354558336, 0.5187764688770438)

In [129]:
# check coefficients
lr.coef_

array([ 0.69011584,  0.92572211, -1.1852778 , 34.66524639, -0.60663964,
        0.45821075,  1.58547655,  3.47134189, 10.74747346])

In [130]:
# create dataframe for coefficients
coefficients = pd.DataFrame(lr.coef_, X_train.columns, columns=['coefs'],)

In [131]:
# view coefficients
coefficients.sort_values(by='coefs', ascending=False)

,coefs
hour,34.665246
Incidences,10.747473
TAVG_CALC,3.471342
SNWD,1.585477
month,0.925722
year,0.690116
SNOW,0.458211
PRCP,-0.606640
day,-1.185278


The most impactful features were hour, traffic incidents, and temperature.

### Support Vector Machines

In [132]:
# instantiate, fit, and score
svr = LinearSVR()
svr.fit(X_train_sc, y_train)
svr.score(X_train_sc, y_train), svr.score(X_test_sc, y_test)

(0.5042374481956952, 0.5097826595604802)

SVM scores did not vary much from linear regression

### AdaBoost Regressor

In [133]:
# instantiate, fit, and score
abr = AdaBoostRegressor()
abr.fit(X_train_sc, y_train)
abr.score(X_train_sc, y_train), abr.score(X_test_sc, y_test)

(0.6922235095638736, 0.691411351252992)

AdaBoost scores improved significantly from linear regression

### Random Forest Regressor

In [11]:
# instantiate, fit, and score
rf = RandomForestRegressor()
rf.fit(X_train_sc, y_train)
rf.score(X_train_sc, y_train), rf.score(X_test_sc, y_test)

(0.9755363535276769, 0.8317865488767585)

Random forest scores are much higher than previous models but is overfit. Will try gridsearching to reduce the overfit.

In [12]:
# params for gridsearching
rf_param = {
    'n_estimators': [200, 250, 300],
    'max_depth': [2, 3, 4, 5]
}
# instantiate, fit, and score
gs = GridSearchCV(rf, param_grid=rf_param, cv=5)
gs.fit(X_train_sc, y_train)
gs.score(X_train_sc, y_train), gs.score(X_test_sc, y_test)

(0.8101708790252456, 0.8109133651091758)

In [40]:
rf = RandomForestRegressor(max_depth=5,n_estimators=200)
rf.fit(X_train_sc, y_train)
filename = 'app-model.pkl'
pickle.dump(rf, open(filename, 'wb'))

In [13]:
print(gs.best_score_)
gs.best_params_

0.805379710921936


{'max_depth': 5, 'n_estimators': 250}

Max_depth of 5 and n_estimators of 200 gives us 81% on both train and test!

In [15]:
preds = gs.predict(X_test_sc)
np.sqrt(mean_squared_error(y_test, preds))

23.629195901946286

The RMSE is 24 calls for the above model.

In [16]:
preds.max()

208.271827315039

In [17]:
preds.min()

78.46769385732357

In [70]:
rf.feature_importances_

array([0.0244732 , 0.02665196, 0.04066642, 0.78730578, 0.01791866,
       0.00116043, 0.00452407, 0.05422635, 0.04307314])

In [71]:
feature_importances = pd.DataFrame(rf.feature_importances_, X_train.columns, columns=['importance'],)

In [72]:
feature_importances.sort_values(by='importance', ascending=False)

,importance
hour,0.787306
TAVG_CALC,0.054226
Incidences,0.043073
day,0.040666
month,0.026652
year,0.024473
PRCP,0.017919
SNWD,0.004524
SNOW,0.001160


Most important features were hour, temperature, traffic incidents, and day.